In [1]:
#Import Libraries
import polars as pl
from dotenv import load_dotenv,find_dotenv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options as chromeoptions
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import requests
import json
import os
import re
from multiprocessing import Pool


In [2]:
#Creating webdriver instance as well as options for fullscreen
options = chromeoptions()
options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(options=options,service=Service(ChromeDriverManager().install()))

In [3]:
#retrieve api key from hidden .env file
load_dotenv(find_dotenv())
hidden_key = os.getenv("hidden_api_key")

In [4]:
#create function to extract data from cryptocompare api
def get_data(symbol,api_key):
    #url created with symbol and api_key taken as global parameters
    url=f"https://min-api.cryptocompare.com/data/v2/histoday?fsym={symbol}&tsym=USD&limit=365&api_key={api_key}"
    #send
    response=requests.get(url)
    data=response.json()["Data"]["Data"]
    data_with_symbol = [{'symbol': symbol, **item} for item in data]
    return data_with_symbol

In [5]:
if __name__ == "__main__":
    symbols=["BTC", "ETH", "LTC"]
    api_key= hidden_key

In [6]:
alls_df =[]
with Pool(4) as pool:
    for symbol in symbols:
        result= get_data(symbol, api_key)
        alls_df.append(pl.DataFrame(result))

In [7]:
df_final=pl.concat(alls_df)

In [9]:
df_final.select(pl.from_epoch(pl.col("time"),time_unit="s")"))

SyntaxError: EOL while scanning string literal (2393537025.py, line 1)

In [11]:
df_final=df_final.drop("volumefrom","volumeto","conversionType","conversionSymbol")

In [ ]:
df_final.append(time_column)

AttributeError: 'DataFrame' object has no attribute 'append'